<div class='alert alert-info' style='text-align: center'><h1>Unsharp Masking Chest X-Rays</h1>
    - yet another chest x-ray processing notebook -
</div>

#### I noticed a lot of the images have various filter convolutions applied to them. Some are blurred, some have unsharp mask. There are some that were *over filtered* and clipping happens.
- Here, I will show how to add an unsharp mask (from scikit), and then histogram equalize the image for a bit of normalization.
- This will create more consistent images across the dataset.

#### The idea is to enhance the contrast and give CNN edge detectors a little more to grab onto.

In [ ]:
# You may need to uncomment and run this command if you intend to load JPEG encoded DICOM files.
#!conda install gdcm -c conda-forge -y

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
from skimage.filters import unsharp_mask
from skimage import exposure

In [ ]:
# This function gets the first image path in a StudyInstanceUID directory in the train set
def get_image_by_study_id(study_id):
    base_path = "/kaggle/input/siim-covid19-detection/"
    study_path = base_path + "train/" + study_id + "/"
    images = []
    for subdir, dirs, files in os.walk(study_path):
        for file in files:     
            image = os.path.join(subdir, file)
            if os.path.isfile(image):
                return image
    return "none"

In [ ]:
# Function to load image
def load_image(study_id):
    img_file = get_image_by_study_id(study_id)
    image = pydicom.dcmread(img_file)
    pixels = image.pixel_array

    min_pixel = np.min(pixels)
    max_pixel = np.max(pixels)

    if image.PhotometricInterpretation == "MONOCHROME1":
        pixels = max_pixel - pixels
    else:
        pixels = pixels
    return pixels

In [ ]:
# Apply unsharp mask and hist equalization
def unsharp_image(pixels):

    # Tweak the radius and amount for more/less sharpening
    unsharp_image = unsharp_mask(pixels, radius=5, amount=2)
    equalized_image = exposure.equalize_hist(unsharp_image)
    
    return equalized_image

In [ ]:
# Display images
def plot_images(pixels, image_unsharp):
    # Plot both images
    fig, axes = plt.subplots(nrows=1, ncols=2,sharex=True, sharey=True, figsize=(12, 12))
    ax = axes.ravel()
    ax[0].imshow(pixels, cmap=plt.cm.gray)
    ax[0].set_title('Original image')
    ax[1].imshow(image_unsharp, cmap=plt.cm.gray)
    ax[1].set_title('After unsharp / hist eq')
    for a in ax:
        a.axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
# Grab an random study ID and get the pixels
pixels = load_image('013d698aeecb')

# Apply unsharp and hist equalization
pixels_unsharp = unsharp_image(pixels)

# Plot the images
plot_images(pixels, pixels_unsharp)

#### As you can tell, the image is more contrasted and seems much clearer.
- Some anatomy like the distal bronchial/vascular markings in the upper lobes are better demonstrated.
- I believe this technique can help with normalization across the dataset and edge detection for CNNs.
- I won't go into the details of unsharp masking. Read more about it here -> https://en.wikipedia.org/wiki/Unsharp_masking
- This should be applied sparingly to images that already have sharpening/unsharp filters.

**Here are some other processing notebooks I made:**
- Lung segmentation without CNN ->https://www.kaggle.com/davidbroberts/lung-segmentation-without-cnn
- Applying filters to x-rays -> https://www.kaggle.com/davidbroberts/applying-filters-to-chest-x-rays
- Rib supression on Chest X-Rays -> https://www.kaggle.com/davidbroberts/rib-suppression-poc
- Manual DICOM VOI LUT -> https://www.kaggle.com/davidbroberts/manual-dicom-voi-lut
- Apply Unsharp Mask to Chest X-Rays -> https://www.kaggle.com/davidbroberts/unsharp-masking-chest-x-rays
- Cropping Chest X-Rays -> https://www.kaggle.com/davidbroberts/cropping-chest-x-rays
- Bounding Boxes on Cropped Images -> https://www.kaggle.com/davidbroberts/bounding-boxes-on-cropped-images
- Visualizing Chest X-Ray bit planes -> https://www.kaggle.com/davidbroberts/visualizing-chest-x-ray-bitplanes
- DICOM full range pixels as CNN input -> https://www.kaggle.com/davidbroberts/dicom-full-range-pixels-as-cnn-input
- Standardizing Chest X-Ray Dataset Exports -> https://www.kaggle.com/davidbroberts/standardizing-cxr-datasets